A base analisada nesse notebook é do Cadastur, que é o sistema de cadastro de pessoas físicas e jurídicas que atuam no setor de turismo, formalizando e legalizando estes prestadores de serviços junto ao Ministério de Turismo.

Com o cadastro, os prestadores de serviços turísticos mantêm uma situação de regularidade no setor e conquistam benefícios como acesso e participação em programas e projetos apoiados ou promovidos pelo Governo Federal, ações de qualificação profissional, acesso a linhas de créditos especiais e visibilidade do seu negócio no site do Cadastur.

O certificado de cadastro tem validade de dois anos para as pessoas jurídicas (obrigatórias ou opcionais), e atesta sua regularidade cadastral.

Pressuposto assumido: ter um certificado vigente ou vencido recentemente* é uma manifestação de comportamento empresarial que caracteriza uma empresa em atividade operante (ou com intenção de operar), ou seja, caracteriza uma empresa existente de fato. (qual a chance de uma empresa inexistente de fato manter ativo um certificado cadastral?)

\* também serão consideradas como empresas existentes de fato aquelas com certificados vencidos em 2020, pressupondo a possibilidade da empresa ficar mais ou menos 1 ano com o certificado vencido, especialmente em uma época de pandemia, em que um dos setores mais prejudicados foi o de turismo.

Portanto, obter a lista dos CNPJs dessas empresas nos permitirá rotular algumas empresas da base nacional como existentes de fato.

Vamos obter os CNPJs das empresas com registro no Cadastur através de downloads de vários arquivos CSV disponíveis no site: http://dados.turismo.gov.br/cadastur

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import glob
import re

In [51]:
# Obtém o html da página web que possui os links de todos os arquivos que precisamos baixar:
response = requests.get('http://dados.turismo.gov.br/cadastur')
soup = BeautifulSoup(response.text, 'html.parser')

In [9]:
# Obtém os elementos <a> que possuem os links para os arquivos CSV que vamos baixar:
lista_tag_a = soup.find_all(lambda tag:tag.name=="a" and '2020' in tag.text and '.CSV' in tag.get('href').upper())
lista_tag_a

[<a href="http://dados.turismo.gov.br/images/csv/Prestadores_Servicos_Turistico_CADASTUR/202001TrimestrePF.csv">2020-1ºTRI</a>,
 <a href="http://dados.turismo.gov.br/images/csv/Prestadores_Servicos_Turistico_CADASTUR/202002TrimestrePF.csv">2020-2ºTRI</a>,
 <a href="http://dados.turismo.gov.br/images/csv/Prestadores_Servicos_Turistico_CADASTUR/202003TrimestrePF.csv">2020-3ºTRI</a>,
 <a href="http://dados.turismo.gov.br/images/csv/cadastur/01/202004TrimestrePF.csv">2020-4ºTRI</a>,
 <a href="http://turismo.gov.br/dadosabertos/cadasturpj/AcampamentoTuristico202001TrimestreCadasturPJ.csv">2020-1ºTRI</a>,
 <a href="http://turismo.gov.br/dadosabertos/cadasturpj/AcampamentoTuristico202002TrimestreCadasturPJ.csv">2020-2ºTRI</a>,
 <a href="http://turismo.gov.br/dadosabertos/cadasturpj/AcampamentoTuristico202003TrimestreCadasturPJ.csv">2020-3ºTRI</a>,
 <a href="http://dados.turismo.gov.br/images/csv/cadastur/02_Acampamento_Turistico/AcampamentoTuristico202004TrimestreCadasturPJ.csv">2020-4ºTRI</a

In [12]:
# Obtém as URLs da página de cada recurso:
lista_url = list()
for tag in lista_tag_a:
    lista_url.append(tag.get('href'))
print(lista_url)
print(len(lista_url))

['http://dados.turismo.gov.br/images/csv/Prestadores_Servicos_Turistico_CADASTUR/202001TrimestrePF.csv', 'http://dados.turismo.gov.br/images/csv/Prestadores_Servicos_Turistico_CADASTUR/202002TrimestrePF.csv', 'http://dados.turismo.gov.br/images/csv/Prestadores_Servicos_Turistico_CADASTUR/202003TrimestrePF.csv', 'http://dados.turismo.gov.br/images/csv/cadastur/01/202004TrimestrePF.csv', 'http://turismo.gov.br/dadosabertos/cadasturpj/AcampamentoTuristico202001TrimestreCadasturPJ.csv', 'http://turismo.gov.br/dadosabertos/cadasturpj/AcampamentoTuristico202002TrimestreCadasturPJ.csv', 'http://turismo.gov.br/dadosabertos/cadasturpj/AcampamentoTuristico202003TrimestreCadasturPJ.csv', 'http://dados.turismo.gov.br/images/csv/cadastur/02_Acampamento_Turistico/AcampamentoTuristico202004TrimestreCadasturPJ.csv', 'http://turismo.gov.br/dadosabertos/cadasturpj/AgenciadeTurismo202001TrimestreCadasturPJ.csv', 'http://turismo.gov.br/dadosabertos/cadasturpj/AgenciadeTurismo202002TrimestreCadasturPJ.csv'

In [14]:
# Faz o download dos CSVs:
for url_csv in lista_url:
    nome_arquivo = url_csv.rsplit('/', 1)[1]
    print('Baixando: ', nome_arquivo, ' de: ', url_csv)
    req = requests.get(url_csv)
    url_content = req.content
    csv_file = open('C:\\DATASETS\\Empresas Existentes de Fato\\Cadastur\\' + nome_arquivo, 'wb')

    csv_file.write(url_content)
    csv_file.close()

Baixando:  202001TrimestrePF.csv  de:  http://dados.turismo.gov.br/images/csv/Prestadores_Servicos_Turistico_CADASTUR/202001TrimestrePF.csv
Baixando:  202002TrimestrePF.csv  de:  http://dados.turismo.gov.br/images/csv/Prestadores_Servicos_Turistico_CADASTUR/202002TrimestrePF.csv
Baixando:  202003TrimestrePF.csv  de:  http://dados.turismo.gov.br/images/csv/Prestadores_Servicos_Turistico_CADASTUR/202003TrimestrePF.csv
Baixando:  202004TrimestrePF.csv  de:  http://dados.turismo.gov.br/images/csv/cadastur/01/202004TrimestrePF.csv
Baixando:  AcampamentoTuristico202001TrimestreCadasturPJ.csv  de:  http://turismo.gov.br/dadosabertos/cadasturpj/AcampamentoTuristico202001TrimestreCadasturPJ.csv
Baixando:  AcampamentoTuristico202002TrimestreCadasturPJ.csv  de:  http://turismo.gov.br/dadosabertos/cadasturpj/AcampamentoTuristico202002TrimestreCadasturPJ.csv
Baixando:  AcampamentoTuristico202003TrimestreCadasturPJ.csv  de:  http://turismo.gov.br/dadosabertos/cadasturpj/AcampamentoTuristico202003Tri

Baixando:  OrganizadoraDeEventos202004TrimestreCadasturPJ.csv  de:  http://dados.turismo.gov.br/images/csv/cadastur/12_Organizador_de_Eventos/OrganizadoraDeEventos202004TrimestreCadasturPJ.csv
Baixando:  PrestadorDeInfraestruturaDeApoioParaEventos202001TrimestreCadasturPJ.csv  de:  http://www.turismo.gov.br/dadosabertos/cadasturpj/PrestadorDeInfraestruturaDeApoioParaEventos202001TrimestreCadasturPJ.csv
Baixando:  PrestadorDeInfraestruturaDeApoioParaEventos202002TrimestreCadasturPJ.csv  de:  http://www.turismo.gov.br/dadosabertos/cadasturpj/PrestadorDeInfraestruturaDeApoioParaEventos202002TrimestreCadasturPJ.csv
Baixando:  PrestadorDeInfraestruturaDeApoioParaEventos202003TrimestreCadasturPJ.csv  de:  http://www.turismo.gov.br/dadosabertos/cadasturpj/PrestadorDeInfraestruturaDeApoioParaEventos202003TrimestreCadasturPJ.csv
Baixando:  PrestadorDeInfraestruturaDeApoioParaEventos202004TrimestreCadasturPJ.csv  de:  http://dados.turismo.gov.br/images/csv/cadastur/13_Prestador_de_Servicos_de_In

Observações:<br>
\- Removido manualmente os arquivos referentes ao cadastro de prestadores pessoas físicas, pois são necessárias apenas as pessoas jurídicas. Removidos: 202001TrimestrePF.csv, 202002TrimestrePF.csv, 202003TrimestrePF.csv e 202004TrimestrePF.csv<br>
\- Não foi possível baixar os arquivos do segundo trimestre de 2020, nem automaticamente e nem manualmente. Como temos arquivos dos demais trimestres, não há problema.

Temos 42 arquivos CSV, correspondentes ao cadastro atualizado em cada trimestre de 2020, separado por setores.
Precisamos agora selecionar apenas os CNPJs com certificado vigente ou vencidos em 2020, e depois consolidar tudo em uma lista única de cnpjs de empresas existentes de fato.

Como não foi encontrado dicionário de dados das bases, foi feita uma análise manual dos arquivos e foram detectadas as seguintes colunas de interesse: CNPJ, Data de Geração do Certificado, Data de Validade do Certificado<br>
Vamos então obtê-las de cada arquivo:

In [31]:
# IMPORTAÇÃO DOS ARQUIVOS CSV:

path = r'C:\DATASETS\Empresas Existentes de Fato\Cadastur'
all_files = glob.glob(path + "/*.csv")

li = []
# Padrão que identifica o CNPJ na linha
pattern = re.compile('(\d{2}\.\d{3}\.\d{3}\/\d{4}\-\d{2});.+;(\d{2}\/\d{2}\/\d{4}[\s\d\:]*?);(\d{2}\/\d{2}\/\d{4}[\s\d\:]*?);')

df = pd.DataFrame(columns=['Arquivo', 'CNPJ', 'Data de Geração do Certificado', 'Data de Validade do Certificado'])

for filename in all_files:
    print('Importando: ', filename)
   
    resultado = list()
    
    with open(filename) as f:
        lines = f.readlines()
        for line in lines:
            if line == "\n" or line.startswith('Razão Social'):
                continue
                
            dados = pattern.search(line)
            if dados and len(dados.groups())==3:
                resultado.append({'Arquivo': filename.rsplit('\\',1)[1], 'CNPJ': dados.groups()[0], 'Data de Geração do Certificado': dados.groups()[1], 'Data de Validade do Certificado': dados.groups()[2]})
            else:
                print('Não encontrou: ', line)
    df = pd.concat([df, pd.DataFrame(resultado)], ignore_index=True)    
    
df.describe                     

Importando:  C:\DATASETS\Empresas Existentes de Fato\Cadastur\AcampamentoTuristico202001TrimestreCadasturPJ.csv
Importando:  C:\DATASETS\Empresas Existentes de Fato\Cadastur\AcampamentoTuristico202003TrimestreCadasturPJ.csv
Importando:  C:\DATASETS\Empresas Existentes de Fato\Cadastur\AcampamentoTuristico202004TrimestreCadasturPJ.csv
Importando:  C:\DATASETS\Empresas Existentes de Fato\Cadastur\AgenciadeTurismo202001TrimestreCadasturPJ.csv
Importando:  C:\DATASETS\Empresas Existentes de Fato\Cadastur\AgenciadeTurismo202003TrimestreCadasturPJ.csv
Importando:  C:\DATASETS\Empresas Existentes de Fato\Cadastur\AgenciadeTurismo202004TrimestreCadasturPJ.csv
Importando:  C:\DATASETS\Empresas Existentes de Fato\Cadastur\CasaDeEspetaculosEquipamentosDeAnimacao202001TrimestreCadasturPJ.csv
Importando:  C:\DATASETS\Empresas Existentes de Fato\Cadastur\CasaDeEspetaculosEquipamentosDeAnimacao202003TrimestreCadasturPJ.csv
Importando:  C:\DATASETS\Empresas Existentes de Fato\Cadastur\CasaDeEspetaculo

<bound method NDFrame.describe of                                                   Arquivo                CNPJ  \
0       AcampamentoTuristico202001TrimestreCadasturPJ.csv  00.466.148/0001-16   
1       AcampamentoTuristico202001TrimestreCadasturPJ.csv  00.735.853/0001-71   
2       AcampamentoTuristico202001TrimestreCadasturPJ.csv  01.489.941/0001-01   
3       AcampamentoTuristico202001TrimestreCadasturPJ.csv  01.566.818/0001-39   
4       AcampamentoTuristico202001TrimestreCadasturPJ.csv  01.579.202/0001-00   
...                                                   ...                 ...   
268285  TransportadoraTuristica202004TrimestreCadastur...  97.755.607/0001-13   
268286  TransportadoraTuristica202004TrimestreCadastur...  97.834.709/0001-24   
268287  TransportadoraTuristica202004TrimestreCadastur...  98.103.757/0001-05   
268288  TransportadoraTuristica202004TrimestreCadastur...  98.106.339/0001-71   
268289  TransportadoraTuristica202004TrimestreCadastur...  98.593.668/0001-

Verificamos que não houve nenhum alerta de 'Não encontrou'. Ou seja, foram recuperados o CNPJ e as datas de todas as linhas de todos os arquivos.<br>
Obs: foi tentada a importação dos CSV usando o método read_csv do Pandas, mas ocorriam erros como 'ParserError: Error tokenizing data. C error: Expected 25 fields in line 99, saw 26', devido à despadronização nos arquivos CSV. Por isso, foi optado pela soluçao acima, que utiliza expressão regular para extração dos dados de interesse.

In [35]:
# Verificando quantidade de CNPJ por arquivo:
display(df.Arquivo.value_counts())
display(df.Arquivo.value_counts().shape)

# Verificando quantidade de CNPJ únicos:
print(df.CNPJ.nunique())

AgenciadeTurismo202004TrimestreCadasturPJ.csv                                           33390
AgenciadeTurismo202003TrimestreCadasturPJ.csv                                           33116
AgenciadeTurismo202001TrimestreCadasturPJ.csv                                           31971
MeiodeHospedagem202004TrimestreCadasturPJ.csv                                           16584
MeiodeHospedagem202003TrimestreCadasturPJ.csv                                           16327
TransportadoraTuristica202004TrimestreCadasturPJ.csv                                    14638
TransportadoraTuristica202001TrimestreCadasturPJ.csv                                    14625
TransportadoraTuristica202003TrimestreCadasturPJ.csv                                    14615
MeiodeHospedagem202001TrimestreCadasturPJ.csv                                           14497
RestaurantesCafeteriaseBares202004TrimestreCadasturPJ.csv                               11606
RestaurantesCafeteriaseBares202003TrimestreCadasturPJ.csv   

(42,)

97291


Verificou-se que foram importados os 42 arquivos CSV existentes. Conforme esperado.

In [33]:
# Verificando tamanhos dos valores na coluna CNPJ
df.CNPJ.str.len().value_counts()

18    268290
Name: CNPJ, dtype: int64

Conforme esperado (definido na expressão regular)

Vamos ver se há datas nulas:

In [36]:
(df.isnull().sum() / df.shape[0] * 100).sort_values(ascending=False)

Data de Validade do Certificado    0.0
Data de Geração do Certificado     0.0
CNPJ                               0.0
Arquivo                            0.0
dtype: float64

Nenhuma data nula.

Vamos selecionar apenas os CNPJs com certificado vigente ou vencido em 2020:

In [45]:
# Verificando situação da data de geração do certificado:
print(df['Data de Geração do Certificado'].str[6:10].value_counts())

# Verificando situação da data de validade:
print(df['Data de Validade do Certificado'].str[6:10].value_counts())

2019    126797
2020    109428
2018     31319
2021       746
Name: Data de Geração do Certificado, dtype: int64
2021    126798
2022    109422
2020     31324
2023       746
Name: Data de Validade do Certificado, dtype: int64


Pela análise da data de validade, cujo ano mínimo é 2020, percebe-se que todos os CNPJs importados estão com o certificado vigente ou vencido em 2020. Ou seja, não é preciso excluir nenhum CNPJ dessa lista. Todos serão considerados como empresas ativas, existentes de fato.

In [48]:
# Vamos ficar apenas com a coluna de cnpj, tirar a formatação e remover os duplicados:
df = df[['CNPJ']]
df.CNPJ = df.CNPJ.str.replace('[\.\-\/]', '', regex=True)
df = df.drop_duplicates()
print(df)

                  CNPJ
0       00466148000116
1       00735853000171
2       01489941000101
3       01566818000139
4       01579202000100
...                ...
267838  73134009000169
267875  74744079000100
268082  90365495000145
268205  94096070000139
268255  96496427000100

[97291 rows x 1 columns]


Salvando o resultdo com a lista de CNPJs de empresas que possuem cadastro ativo no Cadastur:

In [49]:
df.to_csv(r'C:\DATASETS\Empresas Existentes de Fato\prestadores_servicos_Cadastur.csv', index = False)